# Upload Embeddings til Database (Colab)

Denne notebooken genererer embeddings med Google sin `text-multilingual-embedding-002` modell
og laster dem opp til Cloud SQL med pgvector.

**Kjør denne notebooken i Google Colab for enkel autentisering!**

**Forutsetninger:**
- Chunks generert i `chunking.ipynb` (JSONL-filer)
- Database satt opp via Terraform
- Tilgang til GCP-prosjektet `data-science-faggruppe-rag`

In [ ]:
# Installer nødvendige pakker (Colab)
!pip install -q google-genai psycopg2-binary pgvector

print("✅ Pakker installert")

✅ Imports lastet og .env lest


In [ ]:
# Autentiser med Google Cloud (Colab)
from google.colab import auth
auth.authenticate_user()

print("✅ Autentisert med Google Cloud!")

📋 Konfigurasjon:
   - Project: data-science-faggruppe-rag
   - Region: europe-west4
   - Embedding model: text-multilingual-embedding-002 (768 dim)
   - Strategy: metadata

📋 Database:
   - Host: 35.205.154.230
   - Database: vector_db
   - User: vector_db_user


In [ ]:
# Imports
import json
from pathlib import Path
from google import genai
import psycopg2
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector

print("✅ Alle imports lastet")

In [ ]:
# Konfigurasjon
PROJECT_ID = "data-science-faggruppe-rag"
REGION = "europe-west4"

# Embedding-modell konfigurasjon
EMBEDDING_MODEL = "text-multilingual-embedding-002"
EMBEDDING_DIM = 768

# Database-konfigurasjon (Cloud SQL)
DB_HOST = "35.205.154.230"
DB_PORT = "5432"
DB_NAME = "vector_db"
DB_USER = "vector_db_user"
DB_PASSWORD = input("🔐 Database-passord: ")  # Skriv inn passord

# Hvilken chunking-strategi skal lastes opp?
STRATEGY = "metadata"  # Velg: "naive", "overlap", eller "metadata"

print(f"\n📋 Konfigurasjon:")
print(f"   - Project: {PROJECT_ID}")
print(f"   - Region: {REGION}")
print(f"   - Embedding model: {EMBEDDING_MODEL} ({EMBEDDING_DIM} dim)")
print(f"   - Strategy: {STRATEGY}")
print(f"\n📋 Database:")
print(f"   - Host: {DB_HOST}")
print(f"   - Database: {DB_NAME}")
print(f"   - User: {DB_USER}")

## 1. Last opp chunks-filer

Last opp JSONL-filene fra `output/` mappen i ditt lokale prosjekt.

In [ ]:
# Last opp chunks-filer fra din lokale maskin
from google.colab import files

print("📤 Last opp JSONL-filer fra output/ mappen:")
print("   - chunks_naive.jsonl")
print("   - chunks_overlap.jsonl") 
print("   - chunks_metadata.jsonl")
print()

uploaded = files.upload()

print(f"\n✅ Lastet opp {len(uploaded)} filer")

✅ Lastet 160 chunks fra 'metadata' strategi

📝 Eksempel chunk:
{
  "chunk_id": 0,
  "text": "## **ÅRSRUNDSKRIV FOR**",
  "context": "Dette innholdet er i handler om '**ÅRSRUNDSKRIV FOR**'",
  "metadata": {
    "h2": "**ÅRSRUNDSKRIV FOR**"
  },
  "strategy": "metadata",
  "char_count": 23
}


In [ ]:
def load_chunks(strategy: str) -> list[dict]:
    """Last chunks fra JSONL-fil."""
    filepath = f"chunks_{strategy}.jsonl"
    chunks = []
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            chunks.append(json.loads(line))
    return chunks

# Last chunks
chunks = load_chunks(STRATEGY)

print(f"✅ Lastet {len(chunks)} chunks fra '{STRATEGY}' strategi")
print(f"\n📝 Eksempel chunk:")
print(json.dumps(chunks[0], indent=2, ensure_ascii=False)[:500])

## 2. Generer embeddings med Google Vertex AI

In [4]:
# Initialiser Vertex AI client
client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)

print(f"✅ Vertex AI client initialisert")
print(f"   - Project: {PROJECT_ID}")
print(f"   - Region: {REGION}")

✅ Vertex AI client initialisert
   - Project: data-science-faggruppe-rag
   - Region: europe-west4


In [5]:
def get_embeddings_batch(texts: list[str], batch_size: int = 100) -> list[list[float]]:
    """
    Generer embeddings for en liste med tekster i batches.
    
    Args:
        texts: Liste med tekster
        batch_size: Antall tekster per API-kall
    
    Returns:
        Liste med embedding-vektorer
    """
    all_embeddings = []
    total = len(texts)
    
    for i in range(0, total, batch_size):
        batch = texts[i:i + batch_size]
        
        # Generer embeddings for batch
        response = client.models.embed_content(
            model=EMBEDDING_MODEL,
            contents=batch,
            config={'task_type': 'RETRIEVAL_DOCUMENT'}
        )
        
        # Hent ut embeddings
        batch_embeddings = [emb.values for emb in response.embeddings]
        all_embeddings.extend(batch_embeddings)
        
        print(f"   ✓ Prosessert {min(i + batch_size, total)}/{total} tekster")
    
    return all_embeddings

print("📝 Funksjon definert: get_embeddings_batch()")

📝 Funksjon definert: get_embeddings_batch()


In [ ]:
# Hent ut tekst fra chunks
texts = [chunk["text"] for chunk in chunks]

print(f"🔄 Genererer embeddings for {len(texts)} chunks...")
print(f"   (dette kan ta noen minutter)\n")

embeddings = get_embeddings_batch(texts, batch_size=100)

print(f"\n✅ Generert {len(embeddings)} embeddings")
print(f"   - Dimensjoner: {len(embeddings[0])}")
<VSCode.Cell id="#VSC-50278e9a" language="python">
# Verifiser embeddings
print(f"📊 Embedding-statistikk:")
print(f"   - Antall: {len(embeddings)}")
print(f"   - Dimensjoner: {len(embeddings[0])}")
print(f"   - Type: {type(embeddings[0])}")

# Sjekk at alle har samme dimensjon
dims = set(len(e) for e in embeddings)
if len(dims) == 1:
    print(f"   ✅ Alle embeddings har {dims.pop()} dimensjoner")
else:
    print(f"   ⚠️ Varierende dimensjoner: {dims}")

🔄 Genererer embeddings for 160 chunks...
   (dette kan ta noen minutter)



RefreshError: Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.

## 3. Koble til database og last opp

In [ ]:
# Database-funksjoner (standalone for Colab)

def get_db_connection():
    """Opprett database-tilkobling."""
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    register_vector(conn)
    return conn

def test_connection():
    """Test at database-tilkoblingen fungerer."""
    try:
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute("SELECT version();")
        version = cur.fetchone()[0]
        print(f"✅ Tilkoblet database!")
        print(f"   PostgreSQL: {version[:50]}...")
        cur.close()
        conn.close()
        return True
    except Exception as e:
        print(f"❌ Feil ved tilkobling: {e}")
        return False

# Test tilkobling
test_connection()

In [ ]:
def init_embeddings_table(table_name: str = "document_embeddings", embedding_dim: int = 768):
    """Opprett tabell for embeddings hvis den ikke finnes."""
    conn = get_db_connection()
    cur = conn.cursor()
    
    # Aktiver pgvector extension
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    
    # Opprett tabell
    cur.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id SERIAL PRIMARY KEY,
            chunk_id VARCHAR(255),
            text TEXT NOT NULL,
            embedding vector({embedding_dim}) NOT NULL,
            strategy VARCHAR(50),
            metadata JSONB,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """)
    
    # Opprett HNSW indeks for rask søking
    cur.execute(f"""
        CREATE INDEX IF NOT EXISTS {table_name}_embedding_idx 
        ON {table_name} 
        USING hnsw (embedding vector_cosine_ops);
    """)
    
    conn.commit()
    cur.close()
    conn.close()
    print(f"✅ Tabell '{table_name}' klar med {embedding_dim}-dim embeddings")

# Opprett tabell
TABLE_NAME = "document_embeddings"
init_embeddings_table(TABLE_NAME, EMBEDDING_DIM)

init_embeddings_table(
    table_name=TABLE_NAME,
    embedding_dim=EMBEDDING_DIM
)

In [ ]:
def get_table_count(table_name: str = "document_embeddings") -> int:
    """Hent antall rader i tabellen."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute(f"SELECT COUNT(*) FROM {table_name};")
    count = cur.fetchone()[0]
    cur.close()
    conn.close()
    return count

def clear_table(table_name: str = "document_embeddings"):
    """Tøm tabellen."""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute(f"TRUNCATE TABLE {table_name};")
    conn.commit()
    cur.close()
    conn.close()
    print(f"🗑️ Tabell '{table_name}' tømt")

# Valgfritt: Tøm tabellen først (for å unngå duplikater)
# clear_table(TABLE_NAME)

# Sjekk antall rader før insert
count_before = get_table_count(TABLE_NAME)
print(f"📊 Rader i tabellen før insert: {count_before}")

In [ ]:
def insert_chunks_batch(chunks: list, embeddings: list, table_name: str = "document_embeddings", batch_size: int = 100):
    """
    Sett inn chunks med embeddings i batches.
    """
    conn = get_db_connection()
    cur = conn.cursor()
    
    total = len(chunks)
    inserted = 0
    
    for i in range(0, total, batch_size):
        batch_chunks = chunks[i:i + batch_size]
        batch_embeddings = embeddings[i:i + batch_size]
        
        # Forbered data for batch insert
        data = []
        for chunk, embedding in zip(batch_chunks, batch_embeddings):
            data.append((
                chunk.get("chunk_id", f"chunk_{i}"),
                chunk["text"],
                embedding,
                chunk.get("strategy", STRATEGY),
                json.dumps(chunk.get("metadata", {}))
            ))
        
        # Batch insert med execute_values
        execute_values(
            cur,
            f"""
            INSERT INTO {table_name} (chunk_id, text, embedding, strategy, metadata)
            VALUES %s
            """,
            data,
            template="(%s, %s, %s::vector, %s, %s::jsonb)"
        )
        
        inserted += len(batch_chunks)
        print(f"   ✓ Satt inn {inserted}/{total} chunks")
    
    conn.commit()
    cur.close()
    conn.close()
    
    print(f"\n✅ Ferdig! Satt inn {inserted} chunks.")
    return inserted

# Last opp chunks med embeddings
print(f"🔄 Laster opp {len(chunks)} chunks til database...\n")

inserted = insert_chunks_batch(
    chunks=chunks,
    embeddings=embeddings,
    table_name=TABLE_NAME,
    batch_size=100
)

print(f"\n📊 Totalt antall rader i tabellen: {get_table_count(TABLE_NAME)}")

## 4. Verifiser opplasting med test-søk

In [ ]:
def search_similar(query_embedding: list, table_name: str = "document_embeddings", top_k: int = 5):
    """
    Søk etter lignende chunks basert på embedding.
    """
    conn = get_db_connection()
    cur = conn.cursor()
    
    cur.execute(f"""
        SELECT 
            chunk_id,
            text,
            strategy,
            metadata,
            1 - (embedding <=> %s::vector) as similarity
        FROM {table_name}
        ORDER BY embedding <=> %s::vector
        LIMIT %s;
    """, (query_embedding, query_embedding, top_k))
    
    results = []
    for row in cur.fetchall():
        results.append({
            "chunk_id": row[0],
            "text": row[1],
            "strategy": row[2],
            "metadata": row[3],
            "similarity": float(row[4])
        })
    
    cur.close()
    conn.close()
    return results

def get_query_embedding(query: str) -> list[float]:
    """Generer embedding for søkespørring."""
    response = client.models.embed_content(
        model=EMBEDDING_MODEL,
        contents=query,
        config={'task_type': 'RETRIEVAL_QUERY'}
    )
    return response.embeddings[0].values

# Test-søk
test_query = "Hva er dokumentavgift?"

print(f"🔍 Søk: '{test_query}'\n")

query_embedding = get_query_embedding(test_query)
results = search_similar(
    query_embedding=query_embedding,
    table_name=TABLE_NAME,
    top_k=3
)

for i, result in enumerate(results, 1):
    print(f"{'='*80}")
    print(f"Resultat {i} (Similarity: {result['similarity']:.4f})")
    print(f"Strategi: {result['strategy']} | Chunk ID: {result['chunk_id']}")
    print(f"-"*80)
    print(result['text'][:400])
    if len(result['text']) > 400:
        print("...")
    print()

## 5. (Valgfritt) Last opp alle strategier

In [ ]:
# Last opp alle chunking-strategier til databasen
# Dette gir mulighet til å sammenligne strategier i søk

UPLOAD_ALL_STRATEGIES = False  # Sett til True for å laste opp alle

if UPLOAD_ALL_STRATEGIES:
    strategies = ["naive", "overlap", "metadata"]
    
    # Tøm tabellen først
    clear_table(TABLE_NAME)
    
    for strategy in strategies:
        print(f"\n{'='*60}")
        print(f"📦 Prosesserer strategi: {strategy}")
        print(f"{'='*60}")
        
        # Last chunks
        strategy_chunks = load_chunks(strategy)
        print(f"   Lastet {len(strategy_chunks)} chunks")
        
        # Generer embeddings
        texts = [c["text"] for c in strategy_chunks]
        strategy_embeddings = get_embeddings_batch(texts, batch_size=100)
        
        # Last opp
        insert_chunks_batch(
            chunks=strategy_chunks,
            embeddings=strategy_embeddings,
            table_name=TABLE_NAME
        )
    
    print(f"\n✅ Alle strategier lastet opp!")
    print(f"   Totalt: {get_table_count(TABLE_NAME)} rader")
else:
    print("ℹ️ Sett UPLOAD_ALL_STRATEGIES = True for å laste opp alle strategier")

## Ferdig!

Du har nå:
- ✅ Generert embeddings med Google `text-multilingual-embedding-002`
- ✅ Lastet opp chunks til Cloud SQL med pgvector
- ✅ Verifisert med test-søk

**Neste steg:**
- `ranking.ipynb` - Sammenlign retrieval-kvalitet
- `query_rewriting.ipynb` - Forbedre søk med query rewriting
- `eval.ipynb` - Evaluer RAG-systemet